In [2]:
import os

def identify_and_check_scans(root_directory):
    # Tolerance in percentage
    tolerance = 3.0

    # Keywords to search in PAR files and expected file sizes in bytes (PAR, NII, REC)
    scan_keywords_and_sizes = {
        "3D T1": ("PACS 3DT1", [51278, 18350432, 18350080]),
        "fMRI highres": ("fMRI hires", [114451, 8429920, 8429568]),
        "SDDT1": ("fMRI SDDT1 200", [2641255, 103949152, 103948800]),
        "SDDT2": ("fMRI SDDT2 200", [2652305, 104384352, 104384000]),
        "B0-map_RS": ("B0-map_RS", [30503, 9961824, 9961472]),
        "B0-map": ("B0-map", [30498, 9961824, 9961472]),
        "Resting State": ("rsfMRI 140", [2575278, 101350752, 101350400]),
        "SNAT1": ("SNAT1 149", [1781630, 70093152, 70092800]),
        "SNAT2": ("SNAT2 142", [1709480, 67251552, 67251200]),
        "SNAT3": ("SNAT3 141", [1697455, 66777952, 66777600]),
        "DTI A-P": ("jones30_P_NoCardiac", [756906, None, 76185600]),
        "DTI P-A": ("jones30_A_NoCardiac", [756906, None, 76185600])
    }

    # Keep track of maps that are fine and maps that need manual checking
    maps_fine = []
    maps_to_check_manually = []

    # Loop through all subdirectories (maps) in the root directory
    for directory_name in os.listdir(root_directory):
        directory_path = os.path.join(root_directory, directory_name)
        
        # Skip files, only process directories
        if not os.path.isdir(directory_path):
            continue

        print(f"Checking map: {directory_name}")

        identified_scans = {scan_type: [False, False, False] for scan_type in scan_keywords_and_sizes.keys()}
        identified_scans["DTI A-P"][1] = True
        identified_scans["DTI P-A"][1] = True
        abnormal_files = []

        for file_name in os.listdir(directory_path):
            if file_name.endswith(".PAR"):
                file_path = os.path.join(directory_path, file_name)
                with open(file_path, 'r') as file:
                    content = file.read()
                    for scan_type, (keyword, expected_sizes) in scan_keywords_and_sizes.items():
                        if keyword in content:
                            identified_scans[scan_type][0] = True
                            if os.path.exists(os.path.join(directory_path, file_name.replace(".PAR", ".nii"))) and expected_sizes[1]:
                                identified_scans[scan_type][1] = True
                            if os.path.exists(os.path.join(directory_path, file_name.replace(".PAR", ".REC"))):
                                identified_scans[scan_type][2] = True

                            nii_file_name = file_name.replace(".PAR", ".nii")
                            rec_file_name = file_name.replace(".PAR", ".REC")
                            
                            files_with_sizes = [
                                (file_name, os.path.getsize(os.path.join(directory_path, file_name))),
                                (nii_file_name, os.path.getsize(os.path.join(directory_path, nii_file_name))) if identified_scans[scan_type][1] and expected_sizes[1] else (None, None),
                                (rec_file_name, os.path.getsize(os.path.join(directory_path, rec_file_name))) if identified_scans[scan_type][2] else (None, None)
                            ]

                            identified_files = []
                            for (file_name, file_size), expected_size in zip(files_with_sizes, expected_sizes):
                                if expected_size is None or file_name is None:
                                    continue
                                lower_bound = expected_size * (1 - tolerance / 100)
                                upper_bound = expected_size * (1 + tolerance / 100)
                                if lower_bound <= file_size <= upper_bound:
                                    identified_files.append((file_name, file_size, "OK"))
                                else:
                                    identified_files.append((file_name, file_size, "Abnormal"))
                                    abnormal_files.append((file_name, file_size, expected_size))
                            identified_scans[scan_type].extend(identified_files)
                            break

        print("Identified scans:")
        for scan_type, scans in identified_scans.items():
            scans = scans[3:]
            print(scan_type + ":")
            for file_name, file_size, status in scans:
                if file_name:
                    print(f" - {file_name} (Weight: {file_size} bytes) - {status}")

        if abnormal_files:
            print("\nAbnormalities found:")
            for file_name, actual_size, expected_size in abnormal_files:
                print(f" - {file_name}\n   Actual Weight: {actual_size} bytes\n   Expected Weight: {expected_size} bytes")
        else:
            print("No abnormalities were found.")

        # Check for missing scans and formats
        print("\nMissing scans:")
        missing_found = False
        for scan_type, identified_formats in identified_scans.items():
            missing_formats = []
            if not identified_formats[0]: missing_formats.append("PAR")
            if not identified_formats[1] and scan_type not in ["DTI A-P", "DTI P-A"]: missing_formats.append("NII")
            if not identified_formats[2]: missing_formats.append("REC")
            if missing_formats:
                missing_found = True
                print(f" - {scan_type}")
                print(f"   Formats: {', '.join(missing_formats)}")

        if not missing_found:
            print("No scans are missing.")

        if abnormal_files or any(not all(identified_formats[:3]) for identified_formats in identified_scans.values()):
            maps_to_check_manually.append(directory_name)
        else:
            maps_fine.append(directory_name)

    print("\nOverview:")
    print("Maps that are fine:")
    for map_name in maps_fine:
        print(f" - {map_name}")

    print("\nMaps to check manually:")
    for map_name in maps_to_check_manually:
        print(f" - {map_name}")

# Example usage
root_directory = "C:/Users/kovalsa/Documents/export_lab-visit" # Replace this with the path to the root directory containing all maps
identify_and_check_scans(root_directory)


Checking map: SU37009602
Identified scans:
3D T1:
 - SU37009602_2_1.PAR (Weight: 51279 bytes) - OK
 - SU37009602_2_1.REC (Weight: 18350080 bytes) - OK
fMRI highres:
 - SU37009602_3_1.PAR (Weight: 114452 bytes) - OK
 - SU37009602_3_1.nii (Weight: 8429920 bytes) - OK
 - SU37009602_3_1.REC (Weight: 8429568 bytes) - OK
SDDT1:
 - SU37009602_8_1.PAR (Weight: 2639631 bytes) - OK
 - SU37009602_8_1.nii (Weight: 103885152 bytes) - OK
 - SU37009602_8_1.REC (Weight: 103884800 bytes) - OK
SDDT2:
 - SU37009602_9_1.PAR (Weight: 2650031 bytes) - OK
 - SU37009602_9_1.nii (Weight: 104294752 bytes) - OK
 - SU37009602_9_1.REC (Weight: 104294400 bytes) - OK
B0-map_RS:
 - SU37009602_13_1.PAR (Weight: 30504 bytes) - OK
 - SU37009602_13_1.nii (Weight: 9961824 bytes) - OK
 - SU37009602_13_1.REC (Weight: 9961472 bytes) - OK
B0-map:
 - SU37009602_4_1.PAR (Weight: 30499 bytes) - OK
 - SU37009602_4_1.nii (Weight: 9961824 bytes) - OK
 - SU37009602_4_1.REC (Weight: 9961472 bytes) - OK
Resting State:
 - SU37009602_14